# Building a Resource Pack Generator

I'm developing a mod called [FoxNap](https://github.com/OpenBagTwo/FoxNap/) which provides the player with custom musical instruments and music discs. The "killer app" of this mod, besides making all custom items avaialble in survival via villager trades, is that rather than provide a library of music discs, the user can just set the number of music discs to however many they want, and then the mod will dynamically create registrations for that many discs. From there, it's up to the _client_ (read: player) to provide the tracks themselves via a resource pack. Practically what that means is:
- if you're playing single-player, you can add whatever copyrighted music you want to play in your own personal game
- if you're a content creator, you can populate the tracks with your own royalty-free music selections
- if you play on a server, you can populate the tracks with the music that _you_ have the rights to use while your friend populates the tracks with the music that _they_ have the rights to use
- regardless of the situation, you can _swap resource packs_ as the mood strikes, to swap out your music library.

This functionality is there now--the issue is that creating a resource pack from scratch is tedious work, and that's even assuming you've already got everything ready as mono oggs.

So what I'm going to be building out now is a series of steps that will:

1. Convert a mp3, wav, flac or _any_ [`ffmpeg`](https://ffmpeg.org/)-compatible audio format to the correct ogg format
1. Create a custom music disc item texture
1. Populate that disc into a new or existing resource pack, complete with language file entries 

Once I've got the mechanics down, the next step will be creating a cross-platform (I'm gonna have to create a Windows VM 🤢) stand-alone binary (possibly with a _graphical user interface_ 🤮) for distribution with the mod. But that's beyond the scope of this notebook.

## Imports and Setup

In [1]:
# reserved for typing imports

In [2]:
import json
import os
import subprocess
from pathlib import Path

import ffmpeg
from IPython.display import Markdown, display
from PIL import Image

In [3]:
assets_folder = Path("..") / "_static" / "foxnap_rp"

## Converting audio with ffmpeg-python

The conversion process I've been using thusfar involves calling `ffmpeg` from the command line:

```bash
$ ffmpeg -i some_music.mp3 -c:a libvorbis -ac 1 -map a some_music.ogg
```

and I could just keep doing that directly via `subprocess`. But there's also [this handy wrapper](https://github.com/kkroening/ffmpeg-python) I found for ffmpeg, and letting someone else handle the system calls is much more my jam.

The track I'll be working with is ["Higher" by Tobu](https://www.youtube.com/watch?v=blA7epJJaR4), which:
* slaps
* [is available for free download](https://tobu.io/Higher/download)
* is licensed extremely permissively
* contains album art embedded in the mp3, which I'll talk more about in a sec

But to start, let's load the track.

In [4]:
higher = ffmpeg.input(os.fspath(assets_folder / "Tobu - Higher.mp3"))

Since I'm not doing any filters, this should be pretty straightforward. And what I love about this library is that it gives me a really easy way to preview the command I'm about to run.

In [5]:
print(higher.audio.output("higher.ogg", acodec="libvorbis", ac=1).compile())

['ffmpeg', '-i', '../_static/foxnap_rp/Tobu - Higher.mp3', '-map', '0:a', '-ac', '1', '-acodec', 'libvorbis', 'higher.ogg']


I'm following my nose reading their docs and [examples](https://github.com/kkroening/ffmpeg-python/blob/f3079726fae7b7b71e4175f79c5eeaddc1d205fb/examples/README.md), so I hope this is right, but I guess I'll find out in a second. 

In [6]:
higher.audio.output("higher.ogg", acodec="libvorbis", ac=1).overwrite_output().run()

ffmpeg version 5.1.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-16)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnu

Error: ffmpeg error (see stderr output for detail)

Huh. So I'm using a `conda` build of `ffmpeg` in the hopes of easily building a cross-platform app... but it doesn't look like that build ships with libvorbis support:

In [7]:
subprocess.run(["ffmpeg", "-encoders"])

Encoders:
 V..... = Video
 A..... = Audio
 S..... = Subtitle
 .F.... = Frame-level multithreading
 ..S... = Slice-level multithreading
 ...X.. = Codec is experimental
 ....B. = Supports draw_horiz_band
 .....D = Supports direct rendering method 1
 ------
 V....D a64multi             Multicolor charset for Commodore 64 (codec a64_multi)
 V....D a64multi5            Multicolor charset for Commodore 64, extended with 5th color (colram) (codec a64_multi5)
 V..... alias_pix            Alias/Wavefront PIX image
 V..... amv                  AMV Video
 V....D apng                 APNG (Animated Portable Network Graphics) image
 V..... asv1                 ASUS V1
 V..... asv2                 ASUS V2
 V....D libaom-av1           libaom AV1 (codec av1)
 V..... libsvtav1            SVT-AV1(Scalable Video Technology for AV1) encoder (codec av1)
 V....D avrp                 Avid 1:1 10-bit RGB Packer
 V..X.D avui                 Avid Meridien Uncompressed
 V....D ayuv                 Uncompressed p

ffmpeg version 5.1.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-16)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnu

CompletedProcess(args=['ffmpeg', '-encoders'], returncode=0)

It has _vorbis_ but not _libvorbis_, and [vorbis is experimental](https://trac.ffmpeg.org/wiki/TheoraVorbisEncodingGuide).

That being said, it's experimental because:

> native FFmpeg Vorbis audio encoder (`-codec:a vorbis -strict experimental`)... does not provide comparable quality to libvorbis

which is probably fine for these purposes.

In [8]:
higher.audio.output(
    "higher.ogg", acodec="vorbis", ac=1, strict="experimental"
).overwrite_output().run()

ffmpeg version 5.1.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-16)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1662055168074/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnu

Error: ffmpeg error (see stderr output for detail)

>  Current FFmpeg Vorbis encoder only supports 2 channels

Okay, well that's a dealbreaker then.